# 7.3 Problema do Caixeiro Viajante

O problema do caixeiro viajante é um dos mais famosos da computação e da pesquisa operacional. O objetivo é encontrar o ciclo hamiltoniano de menor custo em um grafo completo valorado, onde os vértices representam cidades e as arestas representam o custo de viagem entre elas.

## 7.3.1 Algoritmo Força Bruta

O algoritmo força bruta consiste em enumerar todos os ciclos hamiltonianos possíveis e escolher aquele com menor custo. Apesar de garantir a solução ótima, é inviável para grafos com muitos vértices devido à explosão combinatória (complexidade O(n!)).

In [ ]:
import itertools

def tsp_brute_force(graph, start):
    n = len(graph)
    vertices = list(range(n))
    vertices.remove(start)
    min_path = None
    min_cost = float('inf')
    for perm in itertools.permutations(vertices):
        cost = 0
        k = start
        for j in perm:
            cost += graph[k][j]
            k = j
        cost += graph[k][start]
        if cost < min_cost:
            min_cost = cost
            min_path = (start,) + perm + (start,)
    return min_path, min_cost

# Exemplo de uso:
# Matriz de adjacência para 5 cidades
graph = [
    [0, 2, 9, 10, 7],
    [1, 0, 6, 4, 3],
    [15, 7, 0, 8, 3],
    [6, 3, 12, 0, 11],
    [9, 7, 5, 6, 0]
]
path, cost = tsp_brute_force(graph, 0)
print("Melhor ciclo:", path)
print("Custo:", cost)

## 7.3.2 Heurística do Vizinho Mais Próximo

A heurística do vizinho mais próximo constrói um ciclo hamiltoniano escolhendo sempre a aresta de menor custo para um vértice ainda não visitado. É rápida, mas não garante solução ótima.

In [ ]:
def tsp_nearest_neighbor(graph, start):
    n = len(graph)
    visited = [False] * n
    path = [start]
    total_cost = 0
    current = start
    visited[current] = True
    for _ in range(n - 1):
        next_city = None
        min_cost = float('inf')
        for j in range(n):
            if not visited[j] and 0 < graph[current][j] < min_cost:
                min_cost = graph[current][j]
                next_city = j
        path.append(next_city)
        visited[next_city] = True
        total_cost += min_cost
        current = next_city
    total_cost += graph[current][start]
    path.append(start)
    return path, total_cost

# Exemplo de uso:
path_nn, cost_nn = tsp_nearest_neighbor(graph, 0)
print("Ciclo pelo vizinho mais próximo:", path_nn)
print("Custo:", cost_nn)

## 7.3.3 Heurística da Desigualdade do Triângulo

Para grafos que satisfazem a desigualdade do triângulo, é possível obter uma solução com custo no máximo o dobro do ótimo usando uma abordagem baseada em árvore geradora mínima e percurso Euleriano.

A seguir, um exemplo simplificado dessa heurística:

In [ ]:
import networkx as nx

def tsp_triangle_heuristic(graph):
    n = len(graph)
    G = nx.Graph()
    for i in range(n):
        for j in range(i+1, n):
            G.add_edge(i, j, weight=graph[i][j])
    mst = nx.minimum_spanning_tree(G)
    eulerian_circuit = list(nx.eulerian_circuit(nx.MultiGraph(mst).to_directed()))
    path = []
    visited = set()
    for u, v in eulerian_circuit:
        if u not in visited:
            path.append(u)
            visited.add(u)
    path.append(path[0])
    cost = sum(graph[path[i]][path[i+1]] for i in range(len(path)-1))
    return path, cost

# Exemplo de uso:
path_tri, cost_tri = tsp_triangle_heuristic(graph)
print("Ciclo pela heurística da desigualdade do triângulo:", path_tri)
print("Custo:", cost_tri)

### Passos do Algoritmo Métrico do Caixeiro Viajante

**Entrada:** Um grafo completo valorado Kn que satisfaça a desigualdade do triângulo.

**Passos:**
1. Encontre uma árvore geradora de custo mínimo T do grafo Kn.
2. Crie um grafo euleriano Gₑ duplicando todas as arestas de T.
3. Encontre um ciclo euleriano W em Gₑ.
4. Construa um ciclo hamiltoniano H em G a partir do ciclo W:
   - Percorra a sequência de vértices de W.
   - Sempre que encontrar um vértice já visitado, utilize um "atalho" para o próximo vértice não visitado.
   - Continue até visitar todos os vértices, então feche o ciclo retornando ao inicial.

**Saída:** Um ciclo hamiltoniano.

In [ ]:
import networkx as nx

def tsp_metric_algorithm(graph, start=0):
    n = len(graph)
    G = nx.Graph()
    for i in range(n):
        for j in range(i+1, n):
            G.add_edge(i, j, weight=graph[i][j])
    # 1. Árvore geradora mínima
    mst = nx.minimum_spanning_tree(G)
    # 2. Duplicar arestas da árvore para criar grafo euleriano
    multigraph = nx.MultiGraph()
    multigraph.add_nodes_from(mst.nodes)
    for u, v, data in mst.edges(data=True):
        multigraph.add_edge(u, v, weight=data['weight'])
        multigraph.add_edge(u, v, weight=data['weight'])
    # 3. Encontrar ciclo euleriano
    euler_circuit = list(nx.eulerian_circuit(multigraph, source=start))
    # 4. Construir ciclo hamiltoniano usando atalhos
    hamiltonian_path = []
    visited = set()
    for u, v in euler_circuit:
        if u not in visited:
            hamiltonian_path.append(u)
            visited.add(u)
    # Fechar o ciclo
    hamiltonian_path.append(hamiltonian_path[0])
    total_cost = sum(graph[hamiltonian_path[i]][hamiltonian_path[i+1]] for i in range(len(hamiltonian_path)-1))
    return hamiltonian_path, total_cost

# Exemplo de uso:
path_metric, cost_metric = tsp_metric_algorithm(graph, start=0)
print("Ciclo hamiltoniano pelo algoritmo métrico:", path_metric)
print("Custo:", cost_metric)

## 7.3.4 Aplicações do Problema do Caixeiro Viajante

O problema do caixeiro viajante aparece naturalmente em diversas áreas, especialmente em transportes e logística, mas também em outros contextos devido à simplicidade do seu modelo. Exemplos de aplicações incluem:

- **Programação de tarefas em série:** Dada uma sequência de tarefas a serem executadas por uma máquina, onde o tempo de preparação entre tarefas é relevante, o problema pode ser modelado como um ciclo hamiltoniano de custo mínimo em um grafo dirigido.

- **Perfuração de placas:** Na fabricação de placas de circuito impresso, o deslocamento da broca entre furos pode ser modelado como um problema do caixeiro viajante, buscando minimizar o tempo total de deslocamento.

- **Programação de satélites:** Em missões espaciais, como o Starlight Interferometer Program da NASA, o problema é utilizado para minimizar o uso de combustível ao manobrar satélites para observar diferentes objetos celestes.

- **Programação de rota de entrega:** Muito comum em logística, onde pontos de entrega são vértices e as arestas representam o tempo ou distância entre eles. Um exemplo histórico é a programação de rotas de ônibus escolares, que motivou o estudo do problema.

Essas aplicações mostram a versatilidade do problema do caixeiro viajante em diferentes áreas do conhecimento.